In [7]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time
from requests.exceptions import ConnectionError

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [8]:
base = "C:/Users/valen/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


C:\Users\valen\AppData\Local\Temp\ipykernel_28404\3525900741.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [9]:
df_1.iloc[81917,:]


backers_count                                                              30
blurb                       A horror film about the monsters that make mov...
category                    {"id":297,"name":"Horror","analytics_name":"Ho...
converted_pledged_amount                                               1323.0
country                                                                    US
country_displayable_name                                    the United States
created_at                                                         1580491029
creator                     {"id":967575382,"name":"Logan Fields","slug":"...
currency                                                                  USD
currency_symbol                                                             $
currency_trailing_code                                                   True
current_currency                                                          USD
deadline                                                        

In [10]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list2 = []
risk_list2 = []

In [11]:
block_size = len(df_1) // 100
num_blocks = len(df_1) // 2035
num_blocks
2 * block_size
print(0 * block_size, 1 * block_size)
print(len(slugs))

0 2035
203510


In [ ]:
# instantiate a new Session 
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
           'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
          }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
block_number = 0

headers = {
            'content-type': 'application/json',
            'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
            'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
            'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2'
          }

agents = [
         "Mozilla/5.0 (Symbian/3; Series60/5.2 NokiaN8-00/012.002; Profile/MIDP-2.1 Configuration/CLDC-1.1 ) AppleWebKit/533.4 (KHTML, like Gecko) NokiaBrowser/7.3.0 Mobile Safari/533.4 3gpp-gba",
         "Mozilla/5.0 (Linux; Android 5.0; Lenovo S60-a Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 YaBrowser/17.3.1.383.00 Mobile Safari/E7FBAF",
         "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 244.0.0.12.112 (iPhone12,1; iOS 15_5; en_US; en-US; scale=2.00; 828x1792; 383361019)",
         "Mozilla/5.0 (Linux; U; Android 2.2.1; en-US; GT-S5830 Build/FROYO) AppleWebKit/534.31 (KHTML, like Gecko) UCBrowser/9.0.1.275 U3/0.8.0 Mobile Safari/534.31",
         "Mozilla/5.0 (Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36"
         ]

# pause for 5 seconds every 5 iterations
for i in range(81919, 122106):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(30)
        counter = 0
    counter += 1
    print(f"{i}--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    connect_try = 0
    not_found = True
    while connect_try < 30 and not_found:       
        try:
            r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
                  'User-Agent': "Mozilla/5.0 (Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36",
                  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
                },
                json = {
                    "operationName":"Campaign",
                    "variables":{
                        "slug": slug
                    },
                    "query": query
                })
            not_found = False
        except ConnectionError:
            print("Connection Error")
            time.sleep(5)
            connect_try += 1
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list2.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list2.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list2.append("")
        risk_list2.append("")
        continue

81919--------iceco/iceco-icooler-exclusive-3-in-1-freezer-0------
-------STORY--------
Love to travel outdoor but irritating with melted ice in a traditional cooler? Want to keep food fresh when traveling? Want to buy a refrigerator but keep hesitating because of its high price? ICECO iCooler gets all you want.
ICECO iCooler completely solves your problem! This is a perfect combination of traditonal cooler, refrigerator and freezer. 
It is carefully designed by the team who have won the IDEA and German Reddot Award, manufactured by luxury car refrigerator suppliers. We provide 12 months warranty and US local after-sales service!
Whether you're camping out or playing games at home, iCooler keeps ice cream and cold drinks at your fingertips! Come and prepare for the coming summer!
                                     ✅ 3 in 1 Refrigerator. Fridge/Freezer/Cooler
                                      ✅ PreciseTemperature Control : 0℉-50℉ (-18℃~+10℃)
                                      ✅ 

-------STORY--------
As I started looking for invitations for my own wedding combining up in October, I couldn't find what I was looking for. I decided to try making it myself. I was able to use my computer and various programs to put together several different designs. I loved them all but I had to choose one. Then I realized that I can help other brides win their invitations! If I can help them with their special day, it would mean a lot! 
-------RISKS--------
The challenges I face are getting the website I want up and running. I also need to get my name out there. Advertising. I need the funds to make this happen
81922--------1475732104/help-my-dream-job-come-true------
-------STORY--------
So i am interested in becoming a Professional Photographer and hopefully a fully functional Photographer in the making and follow my dream career as a Portrait Photographer. All i am asking is for a little help with having the opportunity and possibility to raise enough money to buy camera and li

-------STORY--------
Cuatro jóvenes, Santiago, Sofía, Pablo y Rebeca, deciden contactar al papá de Santiago que acaba de fallecer, y el medio por donde lo hacen es atreves de un tablero de Ouija.
Mientras Sofía, la novia de Santiago, teme por contactar con el más allá. Pablo, el mejor amigo, despreocupado reta al Diablo ya que este no existe y la Ouija solo es un juego para él.
Tras haber localizado al padre, este empieza a responder preguntas no coherentes y se dan cuenta que han estado hablando con alguien más. Al momento de querer detener el juego ya es demasiado tarde. La presencia de Dios ha salido del lugar y Satanás los ha invadido, dejando a uno de ellos muerto.
Lo que empezó como un juego terminará con la vida de alguien y el alma de otro, ¿todo lo sucedido será realidad o solamente es parte del juego?
-------RISKS--------
Esta producción tiene una fecha establecida de rodaje, lo cual podría presentar complicaciones, si algún factor natural ,social o político llegara a afectar

-------STORY--------
Halloween Revelation features an ensemble cast of dedicated movie lovers and horror fanatics alike. This film is being made by fans for fans,  a love letter to John Carpenter's Classic that changed the horror genre forever. This fan film seeks to follow in the footsteps of such productions as "Never Hike Alone" and "Dylan's New Nightmare" to take the fan film sub-genre to the next level. 
We are currently in principal photography and with your help we can complete production and move into post production and marketing for the film in order to meet our release date of October 25th, 2020. All proceeds will go directly back into the project to truly make this film the best it can be. 
-------RISKS--------
Challenges: Scheduling, Weather
81928--------butcher2freshmeat/the-butcher-2-fresh-meat------
-------STORY--------
The Butcher 2 Fresh Meat is a sequel to the award winning film The Butcher which has been in 18 film festivals  and has won 8 Awards and counting. This 

-------STORY--------
Welcome to the Magical Skies Enamel Pins campaign! I have created 11 original pin designs of magical places around the world perfect for those who love travelling, fantasy, creatures and pretty pins :)
The initial goal is to unlock the first pin at £300. The rest of the pins will unlock if we reach each of their stretch goals.
You can pledge for any pin you like, even if that one hasn't been unlocked yet. By pledging, you'll be helping to unlock it :)
Pledge now and pay later. You will not be charged until the campaign ends and only then if the initial goal is successful. You can cancel or change your pledge at any time before the campaign ends too!
The Pin Designs
 Freebie Mini Pin
You can add on one of the freebie mini pins to your order for £5 in the add-ons section (in the page after you click pledge).
Vinyl Stickers
You can add on one of the stickers to your order for £2 or the full set for £16 in the add-ons section (in the page after you click pledge).
Art P

...

-------STORY--------
Why did I paint We Are Here To Love?
I felt this was an awesome way to start 2018.  I started painting the 100 pieces of art right after Christmas and for the whole month of January.  2017 was kind of an intense year for a lot of us.  I wanted to start the new year on a positive note - what's more positive than LOVE and making art? 
What are the Make 100 Paintings?
Each painting is on 5x7" canvas. Most of them are acrylic paintings however, I love experimenting and playing with color.  I created a few different abstract varieties.  I will posting random pictures of different styles throughout the campaign.       
You select the series and the paintings will be randomly selected for you.  Once the campaign is completed, I will send an email to everyone, asking for a suggestion for their personal affirmation.   I love writing affirmations for people!!
Humanity needs Love.
My hope is these paintings will remind you to keep your heart open.  No matter what else is happ

<Response [403]>
81978--------975942030/whiskey-raccoons-debut-album------
<Response [403]>
TAKING A NAP ZZZ...
81979--------1646694500/dreamtime------
<Response [403]>
81980--------softbox/softbox------
<Response [403]>
81981--------jonas-bley/eugen-wegner------
<Response [403]>
81982--------studiobenci/weekly-agenda-inspired-by-tuscany------
<Response [403]>
81983--------pinupordiepins/sexy-anime-pins-vol4------
<Response [403]>
TAKING A NAP ZZZ...
81984--------joshuaburnell/into-the-green-on-cd------
<Response [403]>
81985--------1888425537/help-us-fund-hark-the-new-cd-from-wintery-songs-in------
<Response [403]>
81986--------1377040156/amnesiacs-a-horror-web-series------
<Response [403]>
81987--------twowaycrossing1/be-a-part-of-two-way-crossings-next-studio-ep------
<Response [403]>
81988--------letterstoabigail/letters-to-abigails-sophomore-album------
<Response [403]>
TAKING A NAP ZZZ...
81989--------dakotadavehull/the-sacred-and-the-profane------
<Response [403]>
81990--------l

82074--------helperhairbox/helper-hair-box------
<Response [403]>
82075--------countyroadfarms/freeze-dried-100-fruit-and-candy-snacks------
<Response [403]>
82076--------camucrave/tea-bar-vegan-and-eco-friendly-health-bar-flavored-with-tea------
<Response [403]>
82077--------1498804687/salon-lecce-2015------
<Response [403]>
82078--------1140040062/who-shall-we-paint-you-decide-custom-pop-art-canva------
<Response [403]>
TAKING A NAP ZZZ...
82079--------61915613/ayurvedic-skincare------
<Response [403]>
82080--------poroful/motivational-animal-pins------
<Response [403]>
82081--------rejectedcharacters/animal-crossing-inspired-lgbtq-enamel-pins------
<Response [403]>
82082--------404033699/studio-404-art-gallery-studio-storefront-teaching------
<Response [403]>
82083--------970402692/tarms-castle------
<Response [403]>
TAKING A NAP ZZZ...
82084--------397654537/take-a-day-off-of-work-and-make-a-painting------
<Response [403]>
82085--------1734676278/scream-the-online-series------
<Res

<Response [403]>
82173--------joy2020/legacy-the-ultimate-card-game-to-close-the-wealth-gap------
<Response [403]>
TAKING A NAP ZZZ...
82174--------insessionplayingcard/in-session-playing-cards-freshman-year------
<Response [403]>
82175--------creativejuggernaut/clash-0------
<Response [403]>
82176--------3rd-dominion/big-brother-a-custom-foiled-deck------
<Response [403]>
82177--------1076089065/cat-theme-greeting-cards------
<Response [403]>
82178--------coyoteprince/monster-fucker-sticker-series------
<Response [403]>
TAKING A NAP ZZZ...
82179--------merde/merde------
<Response [403]>
82180--------meryljoneswilliams/goldilocks-i-a-16mm-short-film-with-animated-elements------
<Response [403]>
82181--------sidecarpress/sidecar-press-calendar-2017-california-flora-and-f------
<Response [403]>
82182--------941564789/odds-and-socks-a-selection-of-short-stories------
<Response [403]>
82183--------124981153/freehand-arts-anthology------
<Response [403]>
TAKING A NAP ZZZ...
82184--------the

<Response [403]>
82271--------emilioguerra/guelito-guelito-short-film------
<Response [403]>
82272--------732089594/all-bark-and-bite------
<Response [403]>
82273--------411657248/fashions-for-the-gothic-pet-in-your-home------
<Response [403]>
TAKING A NAP ZZZ...
82274--------siteme/gericht-p-one------
<Response [403]>
82275--------toiletseatlidlifter/toilet-seat-lid-lifter-tool------
<Response [403]>
82276--------infinixcnc/infinix-cnc-the-most-affordable-desktop-cnc-machine-kit------
<Response [403]>
82277--------mauryhappy/possible-crowdfunding-tales------
<Response [403]>
82278--------26452851/pioneer-pottery------
<Response [403]>
TAKING A NAP ZZZ...
82279--------lori-yarnismytherapy/just-dying-to-dye-some-yarn------
<Response [403]>
82280--------craftyarchaeologist/make-100-needle-felted-cacti------
<Response [403]>
82281--------andyhillmusic/scoring-the-screen-the-secret-language-of-film-mus------
<Response [403]>
82282--------29561711/return-to-sender-0------
<Response [403]>
8

## Latest Iteration Ran Till: 29029, Local copy latest: 0_29029

In [12]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "data/81919__story" #CHANGE THIS
risk_file_name = "data/81404__risk" #CHANGE THIS 

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list2, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list2, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [31]:
print("risk:", len(risk_list2),"\n", "story:", len(story_list2))

risk: 686 
 story: 686


In [10]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[len(story_list2)-1]['blurb'])

print("LATEST STORY IN LIST", "\n")
print(story_list2[-1])

print("LATEST RISK IN LIST", "\n")
print(risk_list2[-1])

LATEST CALL DF ROW 

A short film about 2 thieves who are blackmailed into digging up dead bodies.
LATEST STORY IN LIST 

We're in the studio recording our second CD, "Boomtown Ghosts", and we are psyched!  Our first CD, "Blue Sky Earth" was a home studio recording, and it was very well received, with nearly 800,000 streams so far through online radio stations.  Our new project is shaping up to be a huge leap forward in every way; from songwriting, singing, musicianship, to excellent production.  It deserves the best we can give, and we think you'll enjoy it! The songs on Boomtown Ghosts are personal, but touch on universal themes that will resonate with many; songs of love and of place, of struggles with a changing world, and with changes within.  Songs that need to be heard! We've been touring and saving for the studio time with the amazing Howlin' Dog Records, but that's just the beginning, and the expenses are adding up faster than we anticipated.  We want to make this as good as p

In [416]:
# story_list2 = story_list2[:-130]
# risk_list2 = risk_list2[:-130]
# story_list2 = story_list2[:-5]
# risk_list2 = risk_list2[:-5]
# headers = {
#             'content-type': 'application/json',
#             'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
#             'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
#             'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2'
#           }
# agents = [
#          "Mozilla/5.0 (Symbian/3; Series60/5.2 NokiaN8-00/012.002; Profile/MIDP-2.1 Configuration/CLDC-1.1 ) AppleWebKit/533.4 (KHTML, like Gecko) NokiaBrowser/7.3.0 Mobile Safari/533.4 3gpp-gba",
#          "Mozilla/5.0 (Linux; Android 5.0; Lenovo S60-a Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 YaBrowser/17.3.1.383.00 Mobile Safari/E7FBAF",
#          "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 244.0.0.12.112 (iPhone12,1; iOS 15_5; en_US; en-US; scale=2.00; 828x1792; 383361019)",
#          "Mozilla/5.0 (Linux; U; Android 2.2.1; en-US; GT-S5830 Build/FROYO) AppleWebKit/534.31 (KHTML, like Gecko) UCBrowser/9.0.1.275 U3/0.8.0 Mobile Safari/534.31",
#          "Mozilla/5.0 (Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36"
#          ]
# headers['user-agent'] = agents[0]

# headers
# story_list[-130]
#story_list[:-130][-1]

# with open("./data/0_4289_story.json", 'r') as f:
#     story_list2 = json.load(f)
# with open("./data/0_4289_risk.json", 'r') as f1:
#     risk_list2 = json.load(f1)

In [165]:
s = requests.Session()

In [421]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

200
This is my fourth solo Kickstarter campaign, and I delivered all three of my previous games in a timely manner. 

I have a complete draft of the text, with the exception of Appendix 1. I have a team of editors lined up who will give me feedback so that I can revise, expand, and polish it as needed. The most time-consuming tasks will be completing the artwork and the layout. I've done this before, with the previous Akashic Titan zine and with my game Project 8Ball. Depending on the stretch goals I will be depending on someone else for illustrations and additional text, but I have worked with Josh before and I am confident he will hit his deadlines. 

I believe I will be able to finish the book by June, but just to be cautious I have set the estimated delivery month as July. 

Barring any unforeseen catastrophes, I should have no problem printing and shipping the zine by July.


In [420]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": slugs[4290]
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
  'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
  'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2h08X3WX0lQSBSeatHjTEHV%2FgAjhqitXq1xvrNIY7MvvonoTMQat4mSbJKrOzGi74PUMwlhlgHz1oDCWy%2BqAiY%2BIQp9ft4c%2BK2ta8%3D--Rds4wbR2FagzmGoy--ZV4JbIYN8J%2FdS8uWWa5EQw%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [255]:
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          }
# GET request to kickstarter to get some token needed for later 
r = s.get("https://www.kickstarter.com", headers=headers)
#BeautifulSoup parses the html returned as a response from the GET request 
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

In [ ]:
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'

In [237]:
print("hello")
time.sleep(3)
print("goodnight")

hello
goodnight
